In [1]:
import pyspark
import pandas as pd
from pyspark.sql import SparkSession

In [22]:
spark.version

'3.0.3'

In [2]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

22/03/01 18:47:52 WARN Utils: Your hostname, DESKTOP-LIPMSCN resolves to a loopback address: 127.0.1.1; using 172.27.8.77 instead (on interface eth0)
22/03/01 18:47:52 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
22/03/01 18:48:03 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/03/01 18:48:04 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [18]:
df = spark.read \
    .option("header", "true") \
    .csv('data/fhvhv_tripdata_2021-02.csv')

In [4]:
df_pandas = pd.read_csv('head.csv')
df_pandas.dtypes

hvfhs_license_num        object
dispatching_base_num     object
pickup_datetime          object
dropoff_datetime         object
PULocationID              int64
DOLocationID              int64
SR_Flag                 float64
dtype: object

In [5]:
spark.createDataFrame(df_pandas).schema

StructType(List(StructField(hvfhs_license_num,StringType,true),StructField(dispatching_base_num,StringType,true),StructField(pickup_datetime,StringType,true),StructField(dropoff_datetime,StringType,true),StructField(PULocationID,LongType,true),StructField(DOLocationID,LongType,true),StructField(SR_Flag,DoubleType,true)))

In [6]:
from pyspark.sql import types

In [8]:
schema = types.StructType([
    types.StructField('hvfhs_license_num', types.StringType(), True),
    types.StructField('dispatching_base_num', types.StringType(), True),
    types.StructField('pickup_datetime', types.TimestampType(), True),
    types.StructField('dropoff_datetime', types.TimestampType(), True),
    types.StructField('PULocationID', types.IntegerType(), True),
    types.StructField('DOLocationID', types.IntegerType(), True),
    types.StructField('SR_Flag', types.StringType(), True)
])

In [19]:
df = spark.read \
    .option("header", "true") \
    .schema(schema) \
    .csv('data/fhvhv_tripdata_2021-02.csv')

In [20]:
df = df.repartition(24)

In [21]:
df.write.parquet('data/fhvhv/2021/01/')

22/03/01 19:05:24 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
22/03/01 19:05:28 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
22/03/01 19:05:31 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
22/03/01 19:05:31 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


In [29]:
df = spark.read.parquet('data/fhvhv/2021/01/*')

In [30]:
df.printSchema()

root
 |-- hvfhs_license_num: string (nullable = true)
 |-- dispatching_base_num: string (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropoff_datetime: timestamp (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- SR_Flag: string (nullable = true)



In [35]:
df.show(5)

+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|hvfhs_license_num|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|
+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|           HV0003|              B02765|2021-02-03 07:11:34|2021-02-03 07:25:03|         197|          76|   null|
|           HV0005|              B02510|2021-02-04 17:39:53|2021-02-04 18:00:26|          36|          80|   null|
|           HV0003|              B02764|2021-02-01 17:14:46|2021-02-01 17:19:17|          86|          86|   null|
|           HV0003|              B02764|2021-02-05 04:28:58|2021-02-05 04:49:17|          80|         173|   null|
|           HV0003|              B02869|2021-02-04 17:33:01|2021-02-04 17:44:37|          80|          37|   null|
+-----------------+--------------------+-------------------+-------------------+

In [48]:
df.registerTempTable('trips')

In [36]:
df = df.withColumn('trip_duration', df.dropoff_datetime - df.pickup_datetime)

In [39]:
df.show(5)

+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+--------------------+
|hvfhs_license_num|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|       trip_duration|
+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+--------------------+
|           HV0003|              B02765|2021-02-03 07:11:34|2021-02-03 07:25:03|         197|          76|   null|13 minutes 29 sec...|
|           HV0005|              B02510|2021-02-04 17:39:53|2021-02-04 18:00:26|          36|          80|   null|20 minutes 33 sec...|
|           HV0003|              B02764|2021-02-01 17:14:46|2021-02-01 17:19:17|          86|          86|   null|4 minutes 31 seconds|
|           HV0003|              B02764|2021-02-05 04:28:58|2021-02-05 04:49:17|          80|         173|   null|20 minutes 19 sec...|
|           HV0003|              B02869|2021-02-

In [46]:
df.select(['pickup_datetime', 'trip_duration']).select(df.pickup_datetime == "2021-02-15").show(5)

ValueError: Could not parse datatype: interval

In [52]:
spark.sql("""SELECT count(*) 
from trips 
where pickup_datetime >= '2021-02-15 00:00:00' and 
pickup_datetime <= '2021-02-15 23:59:59'""").show()

+--------+
|count(1)|
+--------+
|  367170|
+--------+



In [92]:
spark.sql("""SELECT date(pickup_datetime), extract(hour from (dropoff_datetime-pickup_datetime)) as duration 
from trips
order by 2 desc
""").show()

+---------------+--------+
|pickup_datetime|duration|
+---------------+--------+
|     2021-02-11|      20|
|     2021-02-17|      15|
|     2021-02-20|      12|
|     2021-02-03|      11|
|     2021-02-19|      10|
|     2021-02-25|       9|
|     2021-02-18|       9|
|     2021-02-10|       9|
|     2021-02-25|       9|
|     2021-02-10|       9|
|     2021-02-18|       9|
|     2021-02-20|       9|
|     2021-02-05|       8|
|     2021-02-09|       8|
|     2021-02-10|       8|
|     2021-02-03|       8|
|     2021-02-08|       8|
|     2021-02-10|       8|
|     2021-02-02|       8|
|     2021-02-21|       8|
+---------------+--------+
only showing top 20 rows



In [95]:
spark.sql("""SELECT dispatching_base_num, count(*) from trips
group by dispatching_base_num
order by 2 desc
""").show()

+--------------------+--------+
|dispatching_base_num|count(1)|
+--------------------+--------+
|              B02510| 3233664|
|              B02764|  965568|
|              B02872|  882689|
|              B02875|  685390|
|              B02765|  559768|
|              B02869|  429720|
|              B02887|  322331|
|              B02871|  312364|
|              B02864|  311603|
|              B02866|  311089|
|              B02878|  305185|
|              B02682|  303255|
|              B02617|  274510|
|              B02883|  251617|
|              B02884|  244963|
|              B02882|  232173|
|              B02876|  215693|
|              B02879|  210137|
|              B02867|  200530|
|              B02877|  198938|
+--------------------+--------+
only showing top 20 rows



In [98]:
spark.sql("""SELECT PULocationID, DOLocationID, count(*) from trips
where PULocationID != DOLocationID
group by PULocationID, DOLocationID
order by 3 desc
""").show()

+------------+------------+--------+
|PULocationID|DOLocationID|count(1)|
+------------+------------+--------+
|         132|         265|   12542|
|         188|          61|   11814|
|          36|          37|   11491|
|          37|          36|   11487|
|          61|         188|   11462|
|          61|         225|   11342|
|         225|          61|   11293|
|          35|          76|   11244|
|          39|          76|   10688|
|          76|          35|   10621|
|          42|          41|   10304|
|          41|          42|   10260|
|          17|          61|   10185|
|          61|          17|   10181|
|         259|         265|   10091|
|          77|          76|    9748|
|           7|         223|    9465|
|          76|          39|    9338|
|         215|         130|    9288|
|          76|          77|    9162|
+------------+------------+--------+
only showing top 20 rows

